# CSU-K Dev1

## Import & Helperfunctions

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import math
import uuid

In [18]:
def clearY(y):
    clean_input = np.array([]).reshape(0, 1)
    for data in y:
        pos1 = data[0]
        pos2 = data[1]
        pos3 = data[2]
        if  pos1 == 1 and pos2 == 0 and pos3 ==0:
                clean_input = np.vstack((clean_input, [1]))
        else:
                clean_input = np.vstack((clean_input, [0]))
    return clean_input

In [19]:
def evaluate(true_y, pred_y):
    true_classes = []
    for array in true_y:
        if np.array_equal(array,[1, 0, 0]):
            true_classes.append(0)
        elif np.array_equal(array,[0, 1, 0]):
            true_classes.append(1)
        else:
            true_classes.append(2)
        
    CR, CA, PFA, GFA, FR, k = 0, 0, 0, 0, 0, 3.0
    for idx, prediction in enumerate(pred_y):
        # the students answer is correct in meaning and language
        # the system says the same -> accept
        if true_classes[idx] == 0 and prediction == 1:
            CA += 1
        # the system says correct meaning wrong language -> reject
        elif true_classes[idx] == 0 and prediction == 0:
            FR += 1
        # the system says incorrect meaning and incorrect language -> reject
        elif true_classes[idx] == 0 and prediction == 0:
            FR += 1

        # students answer is correct in meaning and wrong in language
        #The system says the same -> reject
        elif true_classes[idx] == 1 and prediction == 0:
            CR += 1
        # the system says correct meaning and correct language -> accept
        elif true_classes[idx] == 1 and prediction == 1:
            PFA += 1
        # the system says incorrect meaning and incorrect language -> reject
        elif true_classes[idx] == 1 and prediction == 0:
            CR += 1

        # students answer is incorrect in meaning and incorrect in language
        # the system says the same -> reject
        elif true_classes[idx] == 2 and prediction == 0:
            CR += 1
        # the system says correct meaning correct language -> accept
        elif true_classes[idx] == 2 and prediction == 1: 
            GFA += 1
        # the system says correct meaning incorrect language -> reject
        elif true_classes[idx] == 2 and prediction == 0:
            CR += 1

    FA = PFA + k * GFA
    Correct = CA + FR
    Incorrect = CR + GFA + PFA
    IncorrectRejectionRate = CR / ( CR + FA + 0.0 )
    CorrectRejectionRate = FR / ( FR + CA + 0.0 )
    # Further metrics
    Z = CA + CR + FA + FR
    Ca = CA / Z
    Cr = CR / Z
    Fa = FA / Z
    Fr = FR / Z
    
    P = Ca / (Ca + Fa)
    R = Ca / (Ca + Fr)
    F = (2 * P * R)/( P + R)
    
    RCa = Ca / (Fr + Ca)
    RFa = Fa / (Cr + Fa)
    
    D = IncorrectRejectionRate / CorrectRejectionRate
    Da = RCa / RFa
    Df = math.sqrt((Da*D))
    
    print('\nINCORRECT UTTERANCES (' + str(Incorrect) + ')' )
    print('CorrectReject    ' + str(CR) )
    print('GrossFalseAccept ' + str(GFA) + '*' + str(k) + ' = ' + str(GFA * k) )
    print('PlainFalseAccept ' + str(PFA) )
    print('RejectionRate    ' + "{:.3f}".format(IncorrectRejectionRate) )

    print('\nCORRECT UTTERANCES (' + str(Correct) + ')')
    print('CorrectAccept    ' + str(CA) )
    print('FalseReject      ' + str(FR) )
    print('RejectionRate    ' + "{:.3f}".format(CorrectRejectionRate) )
    
    print('\n--------------REPORT---------------')
    print('-----------------------------------')
    print('Pr                            ' +  "{:.3f}".format(P) )
    print('F                             ' +  "{:.3f}".format(F) )
    print('Sa                            ' +  "{:.3f}".format(R) )
    #print('R                             ' +  "{:.3f}".format(R) )

    
    print('\n--------------Metrics--------------')
    print('D                             ' +  "{:.3f}".format(D) )
    print('Da                            ' +  "{:.3f}".format(Da) )
    print('Df                            ' +  "{:.3f}".format(Df) )

## Preprocessing

In [20]:
train_x = np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_train_x.csv' ,delimiter=',',usecols=range(11)[1:])
train_y = clearY(np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_train_y.csv', delimiter=',',usecols=range(4)[1:]))

print('Train_x:    ' + str(len(train_x)))
print('Train_y:    ' + str(len(train_y)))

print('\n')


dev_test_x = np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_test_x.csv', delimiter=',',usecols=range(11)[1:])
dev_test_y = np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_test_y.csv', delimiter=',',usecols=range(4)[1:])
print('Dev_Test_x:    ' + str(len(dev_test_x)))
print('Dev_Test_y:    ' + str(len(dev_test_y)))

print('\n')

st2_test_x = np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_st2_test_x.csv', delimiter=',',usecols=range(11)[1:])
st2_test_y = np.loadtxt('/home/spokencall/dnnPaper/expValidation/vec_st2_test_y.csv', delimiter=',',usecols=range(4)[1:])
print('St2_Test_x:    ' + str(len(st2_test_x)))
print('St2_Test_y:    ' + str(len(st2_test_y)))

Train_x:    10092
Train_y:    10092


Dev_Test_x:    2524
Dev_Test_y:    2524


St2_Test_x:    1000
St2_Test_y:    1000


In [21]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [22]:
# Sclae the vectors inorder to get better classification
sc = StandardScaler()
scaled_train_x = sc.fit_transform(train_x)
scaled_dev_test_x = sc.transform(dev_test_x)
scaled_st2_test_x = sc.transform(st2_test_x)

## DNN definition

In [23]:
#Initializing Neural Network
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(64, activation='relu', input_dim=10))
classifier.add(Dropout(0.5))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(1, activation='sigmoid'))

In [24]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
classifier.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

## Training of the model

In [26]:
# Fitting our model 
hist = classifier.fit(scaled_train_x, train_y, batch_size = 5, epochs = 200)

Epoch 1/200
10092/10092 [==============================] - 3s 304us/step - loss: 0.4210 - acc: 0.8378
Epoch 2/200
10092/10092 [==============================] - 3s 294us/step - loss: 0.3811 - acc: 0.8653
Epoch 3/200
10092/10092 [==============================] - 3s 295us/step - loss: 0.3710 - acc: 0.8680
Epoch 4/200
10092/10092 [==============================] - 3s 301us/step - loss: 0.3700 - acc: 0.8713
Epoch 5/200
10092/10092 [==============================] - 3s 299us/step - loss: 0.3625 - acc: 0.8716
Epoch 6/200
10092/10092 [==============================] - 3s 307us/step - loss: 0.3634 - acc: 0.8733
Epoch 7/200
10092/10092 [==============================] - 3s 293us/step - loss: 0.3597 - acc: 0.8724
Epoch 8/200
10092/10092 [==============================] - 3s 303us/step - loss: 0.3561 - acc: 0.8752
Epoch 9/200
10092/10092 [==============================] - 3s 284us/step - loss: 0.3583 - acc: 0.8757
Epoch 10/200
10092/10092 [==============================] - 3s 293us/step - loss: 

10092/10092 [==============================] - 2s 230us/step - loss: 0.3559 - acc: 0.8689
Epoch 160/200
10092/10092 [==============================] - 2s 231us/step - loss: 0.3506 - acc: 0.8729
Epoch 161/200
10092/10092 [==============================] - 2s 233us/step - loss: 0.3476 - acc: 0.8748
Epoch 162/200
10092/10092 [==============================] - 2s 232us/step - loss: 0.3533 - acc: 0.8708
Epoch 163/200
10092/10092 [==============================] - 2s 230us/step - loss: 0.3500 - acc: 0.8745
Epoch 164/200
10092/10092 [==============================] - 2s 227us/step - loss: 0.3464 - acc: 0.8750
Epoch 165/200
10092/10092 [==============================] - 2s 230us/step - loss: 0.3526 - acc: 0.8714
Epoch 166/200
10092/10092 [==============================] - 2s 233us/step - loss: 0.3537 - acc: 0.8699
Epoch 167/200
10092/10092 [==============================] - 2s 231us/step - loss: 0.3520 - acc: 0.8715
Epoch 168/200
10092/10092 [==============================] - 3s 274us/step - l

## Evaluation of the model

### Evaluate model with development test data

In [29]:
dev_y_pred = classifier.predict_classes(scaled_dev_test_x)
evaluate(dev_test_y, dev_y_pred)


INCORRECT UTTERANCES (726)
CorrectReject    626
GrossFalseAccept 23*3.0 = 69.0
PlainFalseAccept 77
RejectionRate    0.811

CORRECT UTTERANCES (1798)
CorrectAccept    1624
FalseReject      174
RejectionRate    0.097

--------------REPORT---------------
-----------------------------------
Pr                            0.918
F                             0.910
Sa                            0.903

--------------Metrics--------------
D                             8.379
Da                            4.776
Df                            6.326
